In [14]:
# Import the standard libraries
import pandas as pd
import numpy as np
import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [15]:
def download_data(sheet_name):
    url = 'https://www.mstatistik-muenchen.de/monatszahlenmonitoring/export/xlsx/mzm_export_alle_monatszahlen.xlsx'
    return pd.read_excel(url, sheet_name=sheet_name)

In [126]:
def convert_time(data):
    # Change format of month column
    for index in data.index:
        item = str(data.MONAT.loc[index])
        data.MONAT.loc[index] = '01' + '/' + item[-2:] + '/' + item[:4]
        data.MONAT.loc[index] = datetime.datetime.strptime(data.MONAT.loc[index],'%d/%m/%Y').strftime('%Y/%m/%d')
    data = data.rename(columns={"MONAT": "DATE", "B": "c"})
    return data.drop(['MONATSZAHL', 'JAHR'], axis=1)

In [128]:
sheets = ['FREIZEIT','KINOS','MUSEEN','ORCHESTER','THEATER','TOURISMUS']
ret = []
for sheet_name in sheets:
    print('Downloading data for: ' + sheet_name + '...')
    # Data download
    # TODO: Download could be accelerated by downloading the Excel file only once.
    df = download_data(sheet_name)

    df = convert_time(df)
    df_clean = df[['AUSPRAEGUNG', 'WERT', 'DATE']]
    df_clean = df_clean[df_clean.WERT != np.NaN]

    # Set set nan values from 2020 to zero
    for index in df_clean.index:
        if datetime.datetime.strptime(df_clean.DATE.loc[index],'%Y/%m/%d') >= datetime.datetime.strptime('2020/01/01','%Y/%m/%d'):
            if np.isnan(df_clean.WERT.loc[index]):
                df_clean.WERT.loc[index] = 0
        else:
            if np.isnan(df_clean.WERT.loc[index]):
                df_clean.WERT.loc[index] = df_clean.WERT.mean()
    
    # Set date as index
    df_clean = df_clean.set_index('DATE')

    # Generate a compund feature table
    for item in df_clean.AUSPRAEGUNG.unique():
        name = item
        tmp = pd.DataFrame(df_clean[df_clean.AUSPRAEGUNG == name].copy().WERT)
        tmp = pd.DataFrame(tmp)
        ret.append(pd.DataFrame(tmp.rename(columns={'WERT': name})))

In [149]:
# Concat the feature table
df_clean = pd.concat(ret, axis=0, sort=True)
df_clean = df_clean.groupby(['DATE'],sort=True).sum()
df_clean = df_clean.reset_index()
df_clean['DATE'] = [datetime.datetime.strptime(date, '%Y/%m/%d').strftime('%d/%b/%Y') for date in df_clean['DATE']]
df_clean = df_clean.set_index('DATE')

In [150]:
df_tmp = df_clean

In [151]:
df_clean

,Alte Pinakothek,Ausland,Außenanlagen Olympiapark (Veranstaltungen),Bayerisches Nationalmuseum,Bayerisches Staatsorchester,Deutsches Museum - Museumsinsel,Deutsches Museum - Verkehrszentrum,Inland,Kleine Olympiahalle,Museum Brandhorst,Museum Mensch und Natur,Münchner Kammerspiele,Münchner Philharmoniker,Münchner Stadtmuseum,Nationaltheater,Neue Pinakothek,Olympia-Eissportzentrum,Olympiahalle,Olympiastadion,Olympiaturm,Pinakothek der Moderne,Prinzregententheater (Großes Haus),Residenztheater,Schackgalerie,Schauburg - Theater für junges Publikum,Städtische Galerie im Lenbachhaus,Theater am Gärtnerplatz,Tierpark Hellabrunn,insgesamt
DATE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000/Jan/01,23178.0,0.0,5.739603e+04,23050.0,2969.100000,61758.0,0.0,0.0,4600.000000,0.0,0.0,0.000000,19295.300000,10637.0,51961.100000,21081.0,53358.000000,40500.000000,6853.0,27558.0,0.000000,0.000000,18730.600000,3830.000000,3423.600000,12687.000000,19525.900000,33033.0,554594.0
2000/Feb/01,20684.0,0.0,5.739603e+04,16000.0,658.800000,59371.0,0.0,0.0,57396.030038,0.0,0.0,0.000000,16545.400000,15769.0,44496.000000,17635.0,38532.000000,85800.000000,172987.0,32847.0,0.000000,0.000000,19830.100000,1553.000000,4783.600000,13433.000000,18896.300000,39591.0,497041.0
2000/Mar/01,26725.0,0.0,6.500000e+03,3827.0,4150.000000,99024.0,0.0,0.0,300.000000,0.0,0.0,0.000000,29405.800000,18051.0,50777.700000,25439.0,39747.000000,62600.000000,244351.0,44306.0,0.000000,0.000000,19615.300000,1333.000000,3889.400000,19288.000000,19523.100000,45113.0,405974.0
2000/Apr/01,16750.0,0.0,1.600000e+03,3491.0,4591.400000,111939.0,0.0,0.0,800.000000,0.0,0.0,0.000000,21083.100000,10351.0,51246.100000,14287.0,30436.000000,17400.000000,348904.0,81363.0,0.000000,0.000000,15786.000000,814.000000,3197.500000,16879.000000,17381.000000,152503.0,413818.0
2000/May/01,21291.0,0.0,1.750000e+03,2243.0,527.600000,91902.0,0.0,0.0,450.000000,0.0,0.0,0.000000,14862.000000,13844.0,45895.500000,14405.0,6039.000000,97100.000000,230485.0,72293.0,0.000000,0.000000,14613.700000,587.000000,5402.000000,14070.000000,18199.200000,82923.0,304364.0
2000/Jun/01,28401.0,0.0,3.900000e+04,5042.0,6150.300000,75860.0,0.0,0.0,1000.000000,0.0,0.0,0.000000,19453.100000,14741.0,28653.600000,18023.0,9668.000000,48000.000000,20423.0,73379.0,0.000000,0.000000,15231.000000,754.000000,3004.800000,12661.000000,15472.100000,100089.0,291474.0
2000/Jul/01,37193.0,0.0,1.015000e+05,10142.0,9342.800000,144578.0,0.0,0.0,57396.030038,0.0,0.0,0.000000,19343.600000,26468.0,57351.700000,23785.0,13580.000000,22500.000000,197930.0,98156.0,0.000000,0.000000,16956.300000,1749.000000,3811.200000,18039.000000,18930.464924,115589.0,429441.0
2000/Aug/01,29980.0,0.0,4.610000e+05,6378.0,11523.119611,110022.0,0.0,0.0,57396.030038,0.0,0.0,0.000000,11498.176135,13660.0,18969.566824,25509.0,22281.000000,4800.000000,293106.0,110355.0,0.000000,0.000000,18949.863599,1264.000000,18940.113252,13161.000000,18930.464924,147673.0,342364.0
2000/Sep/01,30711.0,0.0,1.217000e+05,12439.0,11523.119611,81054.0,0.0,0.0,4500.000000,0.0,0.0,0.000000,27965.500000,12039.0,24402.700000,14703.0,19993.000000,11100.000000,264932.0,79041.0,0.000000,0.000000,1202.300000,1123.000000,18940.113252,12239.000000,6599.800000,87865.0,334245.0


### Copy the dataset

In [166]:
df_clean =df_tmp

In [153]:
df_clean

,Alte Pinakothek,Ausland,Außenanlagen Olympiapark (Veranstaltungen),Bayerisches Nationalmuseum,Bayerisches Staatsorchester,Deutsches Museum - Museumsinsel,Deutsches Museum - Verkehrszentrum,Inland,Kleine Olympiahalle,Museum Brandhorst,Museum Mensch und Natur,Münchner Kammerspiele,Münchner Philharmoniker,Münchner Stadtmuseum,Nationaltheater,Neue Pinakothek,Olympia-Eissportzentrum,Olympiahalle,Olympiastadion,Olympiaturm,Pinakothek der Moderne,Prinzregententheater (Großes Haus),Residenztheater,Schackgalerie,Schauburg - Theater für junges Publikum,Städtische Galerie im Lenbachhaus,Theater am Gärtnerplatz,Tierpark Hellabrunn,insgesamt
DATE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000/Jan/01,23178.0,0.0,5.739603e+04,23050.0,2969.100000,61758.0,0.0,0.0,4600.000000,0.0,0.0,0.000000,19295.300000,10637.0,51961.100000,21081.0,53358.000000,40500.000000,6853.0,27558.0,0.000000,0.000000,18730.600000,3830.000000,3423.600000,12687.000000,19525.900000,33033.0,554594.0
2000/Feb/01,20684.0,0.0,5.739603e+04,16000.0,658.800000,59371.0,0.0,0.0,57396.030038,0.0,0.0,0.000000,16545.400000,15769.0,44496.000000,17635.0,38532.000000,85800.000000,172987.0,32847.0,0.000000,0.000000,19830.100000,1553.000000,4783.600000,13433.000000,18896.300000,39591.0,497041.0
2000/Mar/01,26725.0,0.0,6.500000e+03,3827.0,4150.000000,99024.0,0.0,0.0,300.000000,0.0,0.0,0.000000,29405.800000,18051.0,50777.700000,25439.0,39747.000000,62600.000000,244351.0,44306.0,0.000000,0.000000,19615.300000,1333.000000,3889.400000,19288.000000,19523.100000,45113.0,405974.0
2000/Apr/01,16750.0,0.0,1.600000e+03,3491.0,4591.400000,111939.0,0.0,0.0,800.000000,0.0,0.0,0.000000,21083.100000,10351.0,51246.100000,14287.0,30436.000000,17400.000000,348904.0,81363.0,0.000000,0.000000,15786.000000,814.000000,3197.500000,16879.000000,17381.000000,152503.0,413818.0
2000/May/01,21291.0,0.0,1.750000e+03,2243.0,527.600000,91902.0,0.0,0.0,450.000000,0.0,0.0,0.000000,14862.000000,13844.0,45895.500000,14405.0,6039.000000,97100.000000,230485.0,72293.0,0.000000,0.000000,14613.700000,587.000000,5402.000000,14070.000000,18199.200000,82923.0,304364.0
2000/Jun/01,28401.0,0.0,3.900000e+04,5042.0,6150.300000,75860.0,0.0,0.0,1000.000000,0.0,0.0,0.000000,19453.100000,14741.0,28653.600000,18023.0,9668.000000,48000.000000,20423.0,73379.0,0.000000,0.000000,15231.000000,754.000000,3004.800000,12661.000000,15472.100000,100089.0,291474.0
2000/Jul/01,37193.0,0.0,1.015000e+05,10142.0,9342.800000,144578.0,0.0,0.0,57396.030038,0.0,0.0,0.000000,19343.600000,26468.0,57351.700000,23785.0,13580.000000,22500.000000,197930.0,98156.0,0.000000,0.000000,16956.300000,1749.000000,3811.200000,18039.000000,18930.464924,115589.0,429441.0
2000/Aug/01,29980.0,0.0,4.610000e+05,6378.0,11523.119611,110022.0,0.0,0.0,57396.030038,0.0,0.0,0.000000,11498.176135,13660.0,18969.566824,25509.0,22281.000000,4800.000000,293106.0,110355.0,0.000000,0.000000,18949.863599,1264.000000,18940.113252,13161.000000,18930.464924,147673.0,342364.0
2000/Sep/01,30711.0,0.0,1.217000e+05,12439.0,11523.119611,81054.0,0.0,0.0,4500.000000,0.0,0.0,0.000000,27965.500000,12039.0,24402.700000,14703.0,19993.000000,11100.000000,264932.0,79041.0,0.000000,0.000000,1202.300000,1123.000000,18940.113252,12239.000000,6599.800000,87865.0,334245.0


In [167]:
# Summing up some columns
df_clean['Deutsches Museum - Museumsinsel'] = df_clean['Deutsches Museum - Museumsinsel'] + df_clean['Deutsches Museum - Verkehrszentrum']
df_clean = df_clean.drop(['Deutsches Museum - Verkehrszentrum'],axis=1)
df_clean = df_clean.rename(columns={"Deutsches Museum - Museumsinsel": "Deutsches Museum"})

#df_clean['Außenanlagen Olympiapark (Veranstaltungen)'] = df_clean['Außenanlagen Olympiapark (Veranstaltungen)'] + df_clean['Olympia-Eissportzentrum'] + df_clean['Olympiahalle'] + df_clean['Olympiaturm'] + df_clean['Kleine Olympiahalle']
#df_clean = df_clean.drop(['Olympia-Eissportzentrum', 'Olympiahalle', 'Olympiaturm', 'Kleine Olympiahalle'],axis=1)
#df_clean = df_clean.rename(columns={"Außenanlagen Olympiapark (Veranstaltungen)": "Olympiapark"})

In [165]:
df_clean

,Alte Pinakothek,Ausland,Olympiapark,Bayerisches Nationalmuseum,Bayerisches Staatsorchester,Deutsches Museum,Inland,Museum Brandhorst,Museum Mensch und Natur,Münchner Kammerspiele,Münchner Philharmoniker,Münchner Stadtmuseum,Nationaltheater,Neue Pinakothek,Olympiastadion,Pinakothek der Moderne,Prinzregententheater (Großes Haus),Residenztheater,Schackgalerie,Schauburg - Theater für junges Publikum,Städtische Galerie im Lenbachhaus,Theater am Gärtnerplatz,Tierpark Hellabrunn,insgesamt
DATE,,,,,,,,,,,,,,,,,,,,,,,,
2000/Jan/01,23178.0,0.0,1.834120e+05,23050.0,2969.100000,61758.0,0.0,0.0,0.0,0.000000,19295.300000,10637.0,51961.100000,21081.0,6853.0,0.000000,0.000000,18730.600000,3830.000000,3423.600000,12687.000000,19525.900000,33033.0,554594.0
2000/Feb/01,20684.0,0.0,2.719711e+05,16000.0,658.800000,59371.0,0.0,0.0,0.0,0.000000,16545.400000,15769.0,44496.000000,17635.0,172987.0,0.000000,0.000000,19830.100000,1553.000000,4783.600000,13433.000000,18896.300000,39591.0,497041.0
2000/Mar/01,26725.0,0.0,1.534530e+05,3827.0,4150.000000,99024.0,0.0,0.0,0.0,0.000000,29405.800000,18051.0,50777.700000,25439.0,244351.0,0.000000,0.000000,19615.300000,1333.000000,3889.400000,19288.000000,19523.100000,45113.0,405974.0
2000/Apr/01,16750.0,0.0,1.315990e+05,3491.0,4591.400000,111939.0,0.0,0.0,0.0,0.000000,21083.100000,10351.0,51246.100000,14287.0,348904.0,0.000000,0.000000,15786.000000,814.000000,3197.500000,16879.000000,17381.000000,152503.0,413818.0
2000/May/01,21291.0,0.0,1.776320e+05,2243.0,527.600000,91902.0,0.0,0.0,0.0,0.000000,14862.000000,13844.0,45895.500000,14405.0,230485.0,0.000000,0.000000,14613.700000,587.000000,5402.000000,14070.000000,18199.200000,82923.0,304364.0
2000/Jun/01,28401.0,0.0,1.710470e+05,5042.0,6150.300000,75860.0,0.0,0.0,0.0,0.000000,19453.100000,14741.0,28653.600000,18023.0,20423.0,0.000000,0.000000,15231.000000,754.000000,3004.800000,12661.000000,15472.100000,100089.0,291474.0
2000/Jul/01,37193.0,0.0,2.931320e+05,10142.0,9342.800000,144578.0,0.0,0.0,0.0,0.000000,19343.600000,26468.0,57351.700000,23785.0,197930.0,0.000000,0.000000,16956.300000,1749.000000,3811.200000,18039.000000,18930.464924,115589.0,429441.0
2000/Aug/01,29980.0,0.0,6.558320e+05,6378.0,11523.119611,110022.0,0.0,0.0,0.0,0.000000,11498.176135,13660.0,18969.566824,25509.0,293106.0,0.000000,0.000000,18949.863599,1264.000000,18940.113252,13161.000000,18930.464924,147673.0,342364.0
2000/Sep/01,30711.0,0.0,2.363340e+05,12439.0,11523.119611,81054.0,0.0,0.0,0.0,0.000000,27965.500000,12039.0,24402.700000,14703.0,264932.0,0.000000,0.000000,1202.300000,1123.000000,18940.113252,12239.000000,6599.800000,87865.0,334245.0


In [168]:
# Rename some columns for better clarity
df_clean = df_clean.rename(columns={"insgesamt": "Kinos"})
df_clean = df_clean.rename(columns={"Prinzregententheater (Großes Haus)": "Prinzregententheater"})
df_clean = df_clean.rename(columns={"Ausland": "Ausland (Tourismus)"})
df_clean = df_clean.rename(columns={"Inland": "Inland (Tourismus)"})
df_clean = df_clean.rename(columns={"Außenanlagen Olympiapark (Veranstaltungen)": "Olympiapark"})

# Reset index
df_clean = df_clean.reset_index()

In [169]:
df_clean

,DATE,Alte Pinakothek,Ausland (Tourismus),Olympiapark,Bayerisches Nationalmuseum,Bayerisches Staatsorchester,Deutsches Museum,Inland (Tourismus),Museum Brandhorst,Museum Mensch und Natur,Münchner Kammerspiele,Münchner Philharmoniker,Münchner Stadtmuseum,Nationaltheater,Neue Pinakothek,Olympiastadion,Pinakothek der Moderne,Prinzregententheater,Residenztheater,Schackgalerie,Schauburg - Theater für junges Publikum,Städtische Galerie im Lenbachhaus,Theater am Gärtnerplatz,Tierpark Hellabrunn,Kinos
0,2000/Jan/01,23178.0,0.0,1.834120e+05,23050.0,2969.100000,61758.0,0.0,0.0,0.0,0.000000,19295.300000,10637.0,51961.100000,21081.0,6853.0,0.000000,0.000000,18730.600000,3830.000000,3423.600000,12687.000000,19525.900000,33033.0,554594.0
1,2000/Feb/01,20684.0,0.0,2.719711e+05,16000.0,658.800000,59371.0,0.0,0.0,0.0,0.000000,16545.400000,15769.0,44496.000000,17635.0,172987.0,0.000000,0.000000,19830.100000,1553.000000,4783.600000,13433.000000,18896.300000,39591.0,497041.0
2,2000/Mar/01,26725.0,0.0,1.534530e+05,3827.0,4150.000000,99024.0,0.0,0.0,0.0,0.000000,29405.800000,18051.0,50777.700000,25439.0,244351.0,0.000000,0.000000,19615.300000,1333.000000,3889.400000,19288.000000,19523.100000,45113.0,405974.0
3,2000/Apr/01,16750.0,0.0,1.315990e+05,3491.0,4591.400000,111939.0,0.0,0.0,0.0,0.000000,21083.100000,10351.0,51246.100000,14287.0,348904.0,0.000000,0.000000,15786.000000,814.000000,3197.500000,16879.000000,17381.000000,152503.0,413818.0
4,2000/May/01,21291.0,0.0,1.776320e+05,2243.0,527.600000,91902.0,0.0,0.0,0.0,0.000000,14862.000000,13844.0,45895.500000,14405.0,230485.0,0.000000,0.000000,14613.700000,587.000000,5402.000000,14070.000000,18199.200000,82923.0,304364.0
5,2000/Jun/01,28401.0,0.0,1.710470e+05,5042.0,6150.300000,75860.0,0.0,0.0,0.0,0.000000,19453.100000,14741.0,28653.600000,18023.0,20423.0,0.000000,0.000000,15231.000000,754.000000,3004.800000,12661.000000,15472.100000,100089.0,291474.0
6,2000/Jul/01,37193.0,0.0,2.931320e+05,10142.0,9342.800000,144578.0,0.0,0.0,0.0,0.000000,19343.600000,26468.0,57351.700000,23785.0,197930.0,0.000000,0.000000,16956.300000,1749.000000,3811.200000,18039.000000,18930.464924,115589.0,429441.0
7,2000/Aug/01,29980.0,0.0,6.558320e+05,6378.0,11523.119611,110022.0,0.0,0.0,0.0,0.000000,11498.176135,13660.0,18969.566824,25509.0,293106.0,0.000000,0.000000,18949.863599,1264.000000,18940.113252,13161.000000,18930.464924,147673.0,342364.0
8,2000/Sep/01,30711.0,0.0,2.363340e+05,12439.0,11523.119611,81054.0,0.0,0.0,0.0,0.000000,27965.500000,12039.0,24402.700000,14703.0,264932.0,0.000000,0.000000,1202.300000,1123.000000,18940.113252,12239.000000,6599.800000,87865.0,334245.0
9,2000/Oct/01,37823.0,0.0,2.184540e+05,17788.0,5229.800000,99071.0,0.0,0.0,0.0,0.000000,12040.800000,22603.0,50976.500000,29675.0,219020.0,0.000000,0.000000,20246.900000,4827.000000,3106.700000,28407.000000,18009.500000,72083.0,451179.0


In [ ]:
#Save file
df_clean.to_csv('../data/munich_visitors/munich_visitors.csv', index=False)